https://jira.x5.ru/browse/CVMXC-2801

In [1]:
name = 'CVMXC-2801' + '_' + 'danone' + '_pa'

In [2]:
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/cvm_upgrade/')
sys.path.append('/home/jovyan/cvm_upgrade/upgrade/')
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
sys.path.append('/home/jovyan/x5_some_tasks/Osipov/')
from upgrade.post_analysis2.tools.groups_metrics_aggregation import GroupsMetricsAggregation
from upgrade.post_analysis2.tools.guests_metrics_preprocessing import GuestsMetricsPreprocessing
from upgrade.post_analysis2.tools.post_analysis_params import PostAnalysisParams
from upgrade.post_analysis2.tools.metric_names_mapper import map_col_names_to_russian
from upgrade.post_analysis2.tools.experiment_participants import ExperimentParticipants
from upgrade.post_analysis2.tools.experiment_params import ExperimentParams
import datetime
from datetime import timedelta
from spark import *
from utils_osipov import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [3]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### 1: U-29-Danon-danissimo-postavschik-slip-2.2

In [4]:
campaign_id = 'U-29-Danon-danissimo-postavschik-slip-2.2'
output_fin_report = f'{campaign_id}_fin_report.csv'
output_main_report = f'{campaign_id}_main_report.csv'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        85878: "sms_viber_tg",
        85878: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2022-02-23',
    'date_communication_to': '2022-03-06',

    'wave_ids': [
        -1 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})

post_analysis_params = PostAnalysisParams(**{
    'spark': spark,
    'date_from': '2022-02-23',
    'date_to': '2022-03-06', #дата постпериода
    'control_camp_wave_id': 85878,
    'plu_list': list(map(str, [2058043,2131371,3334525,3388829,3503856,3602086,3666192,4000341,4087605,4134957,
                               4164867,4164884,4164891,49076,78358,4045552,4087605,4087604,4087606,4045551,3388829])),
    'threshold_item_condition': 'plu_list', #есть категория еще
    'count_for_plu_list': 2 #механика от 2-х штук
})


assert post_analysis_params.control_camp_wave_id in experiment_params.map_camp_wave_ids_to_suffixes.keys()

In [5]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [6]:
guest_metrics_preparation = GuestsMetricsPreprocessing(spark, experiment_guests_communications, post_analysis_params)
experiment_guests_with_metrics = guest_metrics_preparation.evaluate()

[Stage 9:(1337 + 12) / 4007][Stage 10:(1260 + 20) / 4007][Stage 11:(349 + 45) / 1221]22/03/10 22:32:06 ERROR cluster.YarnScheduler: Lost executor 38 on mn-hd0249.x5.ru: Unable to create executor due to Unable to register with external shuffle server due to : java.util.concurrent.TimeoutException: Timeout waiting for task.


In [7]:
groups_metrics_aggregation = GroupsMetricsAggregation(spark, experiment_guests_with_metrics, post_analysis_params)
groups_metrics, map_to_effects_table = groups_metrics_aggregation.evaluate()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [8]:
fin_report = map_col_names_to_russian(groups_metrics)
fin_report

,"Размер выборки, чел.","Затраты на коммуникацию и на баллы (сумма delta с учетом redemption), руб.","Затраты на коммуникацию, руб./гость","Затраты на коммуникацию, руб.","Софинансирование, руб./гость","Софинансирование, руб.","Покупка предложения(чек), товаров (шт)","Покупка предложения(категория), товаров (шт)","Покупка предложения(набор plu), товаров (шт)","Гостей, купившие что-нибудь",...,"Эффект Ebitda софинансирование(чек) на группу, руб.",Эффект Ebitda софинансирование(чек) на группу / Ebitda софинансирование(чек) группы,Значимость Ebitda софинансирование(категория),"Эффект Ebitda софинансирование(категория) на гостя, руб.","Эффект Ebitda софинансирование(категория) на группу, руб.",Эффект Ebitda софинансирование(категория) на группу / Ebitda софинансирование(категория) группы,Значимость Ebitda софинансирование(набор plu),"Эффект Ebitda софинансирование(набор plu) на гостя, руб.","Эффект Ebitda софинансирование(набор plu) на группу, руб.",Эффект Ebitda софинансирование(набор plu) на группу / Ebitda софинансирование(набор plu) группы
group_name,,,,,,,,,,,,,,,,,,,,,
1_85878_sms_viber_cg,11396,"-19,636.51",0.00,0.00,0.00,0.00,"378,527.39","5,638.86",148.00,10399,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0_85878_sms_viber_cg,93726,"-153,663.49",0.00,0.00,0.00,0.00,"2,869,073.58","48,322.97","2,309.00",84837,...,"-1,036,708.00",-0.02,0.00,0.10,"9,426.00",0.01,0.00,-0.15,"-14,368.00",-0.08


In [9]:
fin_report['meta_post_analysis'] = str(post_analysis_params.__dict__)
fin_report['meta_experiment_params'] = str(experiment_params.__dict__)
groups_metrics['meta_post_analysis'] = str(post_analysis_params.__dict__)
groups_metrics['meta_experiment_params'] = str(experiment_params.__dict__)

groups_metrics.T.to_csv(output_main_report, encoding='utf-8-sig', sep=';')
fin_report.T.to_csv(output_fin_report, encoding='utf-8-sig', sep=';')

### 2: X-29-41-Danone-Danissimo-Geo2-sof

In [10]:
campaign_id = 'X-29-41-Danone-Danissimo-Geo2-sof'
output_fin_report = f'{campaign_id}_fin_report.csv'
output_main_report = f'{campaign_id}_main_report.csv'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        85657: "sms_viber_tg",
        85657: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2022-02-22',
    'date_communication_to': '2022-03-06',

    'wave_ids': [
        85812 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})

post_analysis_params = PostAnalysisParams(**{
    'spark': spark,
    'date_from': '2022-02-22',
    'date_to': '2022-03-06', #дата постпериода
    'control_camp_wave_id': 85657,
    'plu_list': list(map(str, [2058043,2131371,3334525,3388829,3503856,3602086,3666192,4000341,4087605,4134957,
                               4164867,4164884,4164891,49076,78358,4045552,4087605,4087604,4087606,4045551,3388829])),
    'threshold_item_condition': 'plu_list', #есть категория еще
    'count_for_plu_list': 2 #механика от 2-х штук
})


assert post_analysis_params.control_camp_wave_id in experiment_params.map_camp_wave_ids_to_suffixes.keys()

In [11]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [12]:
guest_metrics_preparation = GuestsMetricsPreprocessing(spark, experiment_guests_communications, post_analysis_params)
experiment_guests_with_metrics = guest_metrics_preparation.evaluate()

In [13]:
groups_metrics_aggregation = GroupsMetricsAggregation(spark, experiment_guests_with_metrics, post_analysis_params)
groups_metrics, map_to_effects_table = groups_metrics_aggregation.evaluate()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [14]:
fin_report = map_col_names_to_russian(groups_metrics)
fin_report

,"Размер выборки, чел.","Затраты на коммуникацию и на баллы (сумма delta с учетом redemption), руб.","Затраты на коммуникацию, руб./гость","Затраты на коммуникацию, руб.","Софинансирование, руб./гость","Софинансирование, руб.","Покупка предложения(чек), товаров (шт)","Покупка предложения(категория), товаров (шт)","Покупка предложения(набор plu), товаров (шт)","Гостей, купившие что-нибудь",...,"Эффект Ebitda софинансирование(чек) на группу, руб.",Эффект Ebitda софинансирование(чек) на группу / Ebitda софинансирование(чек) группы,Значимость Ebitda софинансирование(категория),"Эффект Ebitda софинансирование(категория) на гостя, руб.","Эффект Ebitda софинансирование(категория) на группу, руб.",Эффект Ebitda софинансирование(категория) на группу / Ebitda софинансирование(категория) группы,Значимость Ebitda софинансирование(набор plu),"Эффект Ebitda софинансирование(набор plu) на гостя, руб.","Эффект Ebitda софинансирование(набор plu) на группу, руб.",Эффект Ebitda софинансирование(набор plu) на группу / Ebitda софинансирование(набор plu) группы
group_name,,,,,,,,,,,,,,,,,,,,,
1_85657_sms_viber_cg,90212,"-64,403.84",0.00,0.00,0.00,0.00,"3,510,740.51","61,761.93",731.00,85950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0_85657_sms_viber_cg,767383,"425,587.06",1.36,"1,043,222.94",1.37,"1,054,204.50","29,876,389.96","536,768.82","16,628.00",729921,...,"368,390.00",0.00,0.00,0.32,"241,758.00",0.03,0.00,0.29,"224,962.00",0.27


In [15]:
fin_report['meta_post_analysis'] = str(post_analysis_params.__dict__)
fin_report['meta_experiment_params'] = str(experiment_params.__dict__)
groups_metrics['meta_post_analysis'] = str(post_analysis_params.__dict__)
groups_metrics['meta_experiment_params'] = str(experiment_params.__dict__)

groups_metrics.T.to_csv(output_main_report, encoding='utf-8-sig', sep=';')
fin_report.T.to_csv(output_fin_report, encoding='utf-8-sig', sep=';')

### 3: U-29-Danon-danissimo-postavschik-2.1

In [16]:
campaign_id = 'U-29-Danon-danissimo-postavschik-2.1'
output_fin_report = f'{campaign_id}_fin_report.csv'
output_main_report = f'{campaign_id}_main_report.csv'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        85898: "sms_viber_tg",
        85898: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2022-02-23',
    'date_communication_to': '2022-03-06',

    'wave_ids': [
        85958 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})

post_analysis_params = PostAnalysisParams(**{
    'spark': spark,
    'date_from': '2022-02-23',
    'date_to': '2022-03-06', #дата постпериода
    'control_camp_wave_id': 85898,
    'plu_list': list(map(str, [2058043,2131371,3334525,3388829,3503856,3602086,3666192,4000341,4087605,4134957,
                               4164867,4164884,4164891,49076,78358,4045552,4087605,4087604,4087606,4045551,3388829])),
    'threshold_item_condition': 'plu_list', #есть категория еще
    'count_for_plu_list': 2 #механика от 2-х штук
})


assert post_analysis_params.control_camp_wave_id in experiment_params.map_camp_wave_ids_to_suffixes.keys()

In [17]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [18]:
guest_metrics_preparation = GuestsMetricsPreprocessing(spark, experiment_guests_communications, post_analysis_params)
experiment_guests_with_metrics = guest_metrics_preparation.evaluate()

In [19]:
groups_metrics_aggregation = GroupsMetricsAggregation(spark, experiment_guests_with_metrics, post_analysis_params)
groups_metrics, map_to_effects_table = groups_metrics_aggregation.evaluate()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [20]:
fin_report = map_col_names_to_russian(groups_metrics)
fin_report

,"Размер выборки, чел.","Затраты на коммуникацию и на баллы (сумма delta с учетом redemption), руб.","Затраты на коммуникацию, руб./гость","Затраты на коммуникацию, руб.","Софинансирование, руб./гость","Софинансирование, руб.","Покупка предложения(чек), товаров (шт)","Покупка предложения(категория), товаров (шт)","Покупка предложения(набор plu), товаров (шт)","Гостей, купившие что-нибудь",...,"Эффект Ebitda софинансирование(чек) на группу, руб.",Эффект Ebitda софинансирование(чек) на группу / Ebitda софинансирование(чек) группы,Значимость Ebitda софинансирование(категория),"Эффект Ebitda софинансирование(категория) на гостя, руб.","Эффект Ebitda софинансирование(категория) на группу, руб.",Эффект Ebitda софинансирование(категория) на группу / Ebitda софинансирование(категория) группы,Значимость Ebitda софинансирование(набор plu),"Эффект Ebitda софинансирование(набор plu) на гостя, руб.","Эффект Ebitda софинансирование(набор plu) на группу, руб.",Эффект Ebitda софинансирование(набор plu) на группу / Ebitda софинансирование(набор plu) группы
group_name,,,,,,,,,,,,,,,,,,,,,
0_85898_sms_viber_cg,277457,"105,346.86",1.35,"375,938.70",1.46,"404,149.50","7,447,842.17","120,107.99","6,109.00",250001,...,"-16,781,761.00",-0.14,1.00,-0.62,"-171,288.00",-0.09,0.00,0.14,"40,038.00",0.11
1_85898_sms_viber_cg,73048,"-74,249.45",0.00,0.00,0.00,0.00,"2,342,279.61","36,194.96",948.00,66768,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
fin_report['meta_post_analysis'] = str(post_analysis_params.__dict__)
fin_report['meta_experiment_params'] = str(experiment_params.__dict__)
groups_metrics['meta_post_analysis'] = str(post_analysis_params.__dict__)
groups_metrics['meta_experiment_params'] = str(experiment_params.__dict__)

groups_metrics.T.to_csv(output_main_report, encoding='utf-8-sig', sep=';')
fin_report.T.to_csv(output_fin_report, encoding='utf-8-sig', sep=';')

### 4: U-29-Danon-danissimo-postavschik-2

In [22]:
campaign_id = 'U-29-Danon-danissimo-postavschik-2'
output_fin_report = f'{campaign_id}_fin_report.csv'
output_main_report = f'{campaign_id}_main_report.csv'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        85217: "sms_viber_tg",
        85224: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2022-02-21',
    'date_communication_to': '2022-03-06',

    'wave_ids': [
        85572 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})

post_analysis_params = PostAnalysisParams(**{
    'spark': spark,
    'date_from': '2022-02-21',
    'date_to': '2022-03-06', #дата постпериода
    'control_camp_wave_id': 85224,
    'plu_list': list(map(str, [2058043,2131371,3334525,3388829,3503856,3602086,3666192,4000341,4087605,4134957,
                               4164867,4164884,4164891,49076,78358,4045552,4087605,4087604,4087606,4045551,3388829])),
    'threshold_item_condition': 'plu_list', #есть категория еще
    'count_for_plu_list': 2 #механика от 2-х штук
})


assert post_analysis_params.control_camp_wave_id in experiment_params.map_camp_wave_ids_to_suffixes.keys()

In [23]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [24]:
guest_metrics_preparation = GuestsMetricsPreprocessing(spark, experiment_guests_communications, post_analysis_params)
experiment_guests_with_metrics = guest_metrics_preparation.evaluate()

In [25]:
groups_metrics_aggregation = GroupsMetricsAggregation(spark, experiment_guests_with_metrics, post_analysis_params)
groups_metrics, map_to_effects_table = groups_metrics_aggregation.evaluate()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [26]:
fin_report = map_col_names_to_russian(groups_metrics)
fin_report

,"Размер выборки, чел.","Затраты на коммуникацию и на баллы (сумма delta с учетом redemption), руб.","Затраты на коммуникацию, руб./гость","Затраты на коммуникацию, руб.","Софинансирование, руб./гость","Софинансирование, руб.","Покупка предложения(чек), товаров (шт)","Покупка предложения(категория), товаров (шт)","Покупка предложения(набор plu), товаров (шт)","Гостей, купившие что-нибудь",...,"Эффект Ebitda софинансирование(чек) на группу, руб.",Эффект Ebitda софинансирование(чек) на группу / Ebitda софинансирование(чек) группы,Значимость Ebitda софинансирование(категория),"Эффект Ebitda софинансирование(категория) на гостя, руб.","Эффект Ebitda софинансирование(категория) на группу, руб.",Эффект Ebitda софинансирование(категория) на группу / Ebitda софинансирование(категория) группы,Значимость Ebitda софинансирование(набор plu),"Эффект Ebitda софинансирование(набор plu) на гостя, руб.","Эффект Ebitda софинансирование(набор plu) на группу, руб.",Эффект Ebitda софинансирование(набор plu) на группу / Ebitda софинансирование(набор plu) группы
group_name,,,,,,,,,,,,,,,,,,,,,
0_85217_sms_viber_tg,177331,"-286,120.22",1.35,"240,270.06",1.46,"258,399.00","6,731,942.18","130,022.56","8,931.00",164229,...,"-20,369,056.00",-0.18,1.00,-0.90,"-159,514.00",-0.07,1.00,0.64,"113,053.00",0.18
0_85224_sms_viber_cg,59250,"-156,064.60",0.00,0.00,0.00,0.00,"2,793,765.88","50,484.29","2,081.00",55784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
fin_report['meta_post_analysis'] = str(post_analysis_params.__dict__)
fin_report['meta_experiment_params'] = str(experiment_params.__dict__)
groups_metrics['meta_post_analysis'] = str(post_analysis_params.__dict__)
groups_metrics['meta_experiment_params'] = str(experiment_params.__dict__)

groups_metrics.T.to_csv(output_main_report, encoding='utf-8-sig', sep=';')
fin_report.T.to_csv(output_fin_report, encoding='utf-8-sig', sep=';')

22/03/12 10:03:26 ERROR cluster.YarnClientSchedulerBackend: YARN application has exited unexpectedly with state KILLED! Check the YARN application logs for more details.
22/03/12 10:03:26 ERROR cluster.YarnClientSchedulerBackend: Diagnostics message: Application killed by user.
